In [45]:
import os
import sys
import geopandas
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt

In [46]:
#data_folder = "C:\\Users\\Lena\\Dropbox\\Hackathon\\rv"

In [47]:
data_folder = "/Users/lena/Dropbox/Hackathon/rv"

In [48]:
OSM_path = os.path.join(
    data_folder, 'incoming_data', 'multipolygons.shp')

buildings_path = os.path.join(
    data_folder, 'data','buildings', 'stluc_buildings.shp')

OSM_extracted_path = os.path.join(
    data_folder, 'data','buildings', 'OSM_socials.shp')

buildings = geopandas.read_file(buildings_path)
OSM = geopandas.read_file(OSM_path)

In [49]:
def lookup(lookup_dictionary, x):
    if  x in lookup_dictionary.keys():
        return lookup_dictionary[x]
    else:
        return None

### DEFINE DICTIONARIES

In [50]:
amenity_lookup_dictionary = {
                            'school':'education',
                            'library':'education',
                            'university':'education',
                            'college':'education',
                            'pre_school':'education',
                            'childcare': 'education',
                            'kindergarden':'education',
                            'dentist':'healthcare', 
                            'hospital': 'healthcare', 
                            'pharmacy': 'healthcare', 
                            'doctors': 'healthcare', 
                            'clinic': 'healthcare', 
                            'fast_food':'food',
                            'restaurant':'food',
                            'cafe':'food',
                            'marketplace':'food',
                            'bank':'finance',
                            'money_transfer':'finance',
                            'youth_centre':'emergency', 
                            'social_facility':'emergency',
                            'courthouse':'civic',
                            'prison':'civic',
                            'bus_station':'transport',
                            'parking':'transport',
                            'restaurant':'tourism',
                            'townhall':'civic',
                            'fuel':'fuel'
                            }

office_lookup_dictionary = {
                            'government': 'civic'
                            }
tourism_lookup_dictionary = {
                            'hotel': 'tourism'
                            }
shop_lookup_dictionary = {
                        'convenience':'food',
                        'supermarket':'food',
                        'mall': 'food',
                        'butcher':'food',
                        'bakery':'food'
                        }

In [51]:
#apply lookup to amenity and add new category 
OSM['category'] = OSM['amenity'].apply(lambda x: lookup(amenity_lookup_dictionary, x))
education = OSM[OSM.category == 'education']
healthcare = OSM[OSM.category == 'healthcare']
emergency = OSM[OSM.category == 'emergency']
food = OSM[OSM.category == 'food']
civic = OSM[OSM.category == 'civic']
transport = OSM[OSM.category == 'transport']
tourism = OSM[OSM.category == 'tourism']
finance = OSM[OSM.category == 'finance']
fuel = OSM[OSM.category == 'fuel']

OSM['category'] = OSM['office'].apply(lambda x: lookup(office_lookup_dictionary, x))
civic = OSM[OSM.category == 'civic']

OSM['category'] = OSM['tourism'].apply(lambda x: lookup(tourism_lookup_dictionary, x))
tourism = tourism.append(OSM[OSM.category == 'tourism'])

OSM['category'] = OSM['shop'].apply(lambda x: lookup(shop_lookup_dictionary, x))
food = food.append(OSM[OSM.category == 'food'])


In [52]:
social_ids = [education, healthcare, emergency, food, civic, finance, fuel]

for social_id in social_ids:
    print(len(social_id))

121
45
4
194
29
25
26


In [53]:
type(fuel)

geopandas.geodataframe.GeoDataFrame

In [55]:
fuel = fuel.reset_index()


In [56]:
fuel = fuel.rename(columns={
    'index':'id', 

    'amenity':'type'})

In [57]:
fuel = fuel[[
    'id', 
    'name',
    'type', 
    'geometry'
]]


In [60]:
fuel = geopandas.GeoDataFrame(fuel)
fuel.crs = {'init' :'epsg:4326'}
fuel = fuel.to_crs({'init': 'epsg:2006'})

In [61]:
fuel.to_file(os.path.join(
    data_folder, 'data','buildings', 'fuel.shp'))

### Export all OSM social data

In [11]:
#social_ids = [education, healthcare]
social_OSM_merged = []
#social_OSM_merged = pd.DataFrame(columns =['category','geometry'])
for social_id in social_ids:
    social_df = geopandas.GeoDataFrame(social_id[[
            'category',
            'geometry'
        ]])
    social_OSM_merged.append(social_df)
social_OSM_merged = pd.concat(social_OSM_merged, axis=0, sort=False)


social_OSM_merged = social_OSM_merged.reset_index()
social_OSM_merged = social_OSM_merged.rename(columns ={
    'index' : 'OSM_id',
    'category':'type'
})
social_OSM_merged.crs = {'init' :'epsg:4326'}
social_OSM_merged = social_OSM_merged.to_crs({'init': 'epsg:2006'})
social_OSM_merged.to_file(OSM_extracted_path)
social_OSM_merged.head()

,OSM_id,type,geometry
0,215,education,"POLYGON ((513586.4550624012 1517604.187842278,..."
1,352,education,"POLYGON ((509246.9768230459 1548504.102553842,..."
2,355,education,"POLYGON ((509352.9349526085 1548803.034802738,..."
3,358,education,"POLYGON ((508023.0197717446 1550133.141026469,..."
4,489,education,"POLYGON ((513574.364330921 1517709.194154475, ..."


# Merge all point assets to OSM socials 

In [12]:
social_point_ids = ['healthcare_points', 'emergency_points', 'food_points', 'tourism_points', 'education_points', 'fuel_points']

In [13]:
def load_social(data_folder, social_id):  
    social_path = os.path.join(
        data_folder, 'data','buildings', '{}.shp'.format(social_id))
    print("Reading:", social_path)  
    socials = geopandas.read_file(social_path)
    
    if socials.crs != {'init':'epsg:2006'}:
        socials = socials.to_crs({'init':'epsg:2006'})
    return socials

In [14]:
social_point_path = os.path.join(
    data_folder, 'data', 'buildings', 'social_points_merged.shp')

## Merge all social point assets together with consistent naming

In [17]:
social_point_all =[]
for social_point_id in social_point_ids:
    social_point_df = load_social(data_folder, social_point_id)
    print(len(social_point_df))
    social_point_all.append(social_point_df)
    
social_point_all = pd.concat(social_point_all, axis=0, sort = False, ignore_index=True)
social_point_all.reset_index(inplace= True)
social_point_all = social_point_all.rename(columns={
    'index': 'id_soc_point'
})
social_point_all.to_file(social_point_path)
social_point_all.head()

Reading: /Users/lena/Dropbox/Hackathon/rv/data/buildings/healthcare_points.shp
78
Reading: /Users/lena/Dropbox/Hackathon/rv/data/buildings/emergency_points.shp
8
Reading: /Users/lena/Dropbox/Hackathon/rv/data/buildings/food_points.shp
35
Reading: /Users/lena/Dropbox/Hackathon/rv/data/buildings/tourism_points.shp
45
Reading: /Users/lena/Dropbox/Hackathon/rv/data/buildings/education_points.shp
122
Reading: /Users/lena/Dropbox/Hackathon/rv/data/buildings/fuel_points.shp


DriverError: /Users/lena/Dropbox/Hackathon/rv/data/buildings/fuel_points.shp: No such file or directory

In [18]:
social_point_all.index

<function list.index(value, start=0, stop=9223372036854775807, /)>

--- CHECK NEW OSM SOCIALS COMBINE FILE ---

### Determine all intersections: assign polygon geometry to social point assets; keep all OSM data and add the social ones
288 social points
418 OSM social assets 

86 inner join 

In [19]:
OSM_point_intersect = geopandas.sjoin(social_OSM_merged,social_point_all,  how='left', op='intersects')

OSM_point_intersect = OSM_point_intersect.rename(columns={
    'type_left':'type'
})

OSM_point_intersect = OSM_point_intersect[[
    'type',
    'OSM_id',
    'id_soc_point',
    'name',
    'geometry'
]]
OSM_point_intersect.drop_duplicates(['id_soc_point'])
OSM_point_intersect

ValueError: 'right_df' should be GeoDataFrame, got <class 'list'>

In [20]:
OSM_point_intersect = geopandas.sjoin(social_OSM_merged, OSM_point_intersect, how='left')
len(OSM_point_intersect)

NameError: name 'OSM_point_intersect' is not defined

In [21]:
OSM_point_intersect.head()
OSM_point_intersect = OSM_point_intersect[[
    'OSM_id_left',
    'type_left',
    'id_soc_point',
    'name',
    'geometry'
]]

NameError: name 'OSM_point_intersect' is not defined

In [22]:
OSM_point_intersect = geopandas.sjoin(social_OSM_merged, OSM_point_intersect, how='inner')
len(OSM_point_intersect)

NameError: name 'OSM_point_intersect' is not defined

In [30]:
OSM_point_intersect = OSM_point_intersect.rename(columns={
    'OSM_id_left':'OSM_id',
    'type_left':'type'
})

### Merge missing point assets to OSM for overview with both polygons and points -> by adding based on id those that are merged and adding additional ones 

In [23]:
OSM_point_all = OSM_point_intersect.merge(social_point_all,how ='outer', on = ['id_soc_point','type', 'name'])
OSM_point_all

NameError: name 'OSM_point_intersect' is not defined

In [24]:
OSM_socials_point_all_path = os.path.join(
    data_folder, 'data', 'buildings', 'OSM_social_points_merged.csv')

In [25]:
OSM_point_all.to_csv(OSM_socials_point_all_path)

NameError: name 'OSM_point_all' is not defined

OSM_social_point_manualedit = pd.read_csv(OSM_socials_point_all_path)

OSM_social_point_manualedit_gdf = geopandas.GeoDataFrame(OSM_social_point_manualedit)

OSM_socials_point_all_manualedit_path = os.path.join(
    data_folder, 'data', 'buildings', 'OSM_social_points_merged.shp')


OSM_social_point_manualedit_gdf.to_file(OSM_socials_point_all_manualedit_path)

# Merge buildings and social OSM data together 

### Load other social infrastructure point shapefiles

socials_point_non_OSM_gdf = geopandas.GeoDataFrame(socials_point_non_OSM)
socials_point_non_OSM_gdf.head()

socials_point_non_OSM_path = os.path.join(
    data_folder, 'data', 'buildings', 'social_non_OSM.csv')

socials_point_non_OSM = pd.read_csv(socials_point_non_OSM_path)

socials_point_non_OSM.head()


socials_point_non_OSM_gdf= social_point_all.merge(socials_point_non_OSM, on=['id_soc_point', 'name', 'type'])
socials_point_non_OSM_gdf.head()


## Merge all social points with closest building

def find_nearest(buildings, point):
    distances = buildings.geometry.distance(point.geometry)
    min_dist = None
    nearest_id = None
    for building_id, distance in zip(buildings.id, distances):
        if min_dist is None or distance < min_dist:
            min_dist = distance
            nearest_id = building_id
    return nearest_id

In [ ]:
 # find nearest building to each point
nearest_ids = []
for social_point in social_point_all.itertuples():
    nearest_id = find_nearest(buildings,social_point_all)
    nearest_ids.append(nearest_id)

social_point_all['nearest_building_id'] = nearest_ids


social_point_all

links = social_point_all[['nearest_building_id','id_soc_point']]
links.head()

OSM_socials_point_all_manualedit_path = os.path.join(
    data_folder, 'data', 'buildings', 'OSM_social_points_rest.shp')


socials_point_non_OSM_gdf.to_file(OSM_socials_point_all_manualedit_path)

def load_social(data_folder, social_id):  
    social_path = os.path.join(
        data_folder, 'incoming_data', '{}.shp'.format(social_id))
    print("Reading:", social_path)  
    socials = geopandas.read_file(social_path)
    
    if socials.crs != {'init':'epsg:2006'}:
        socials = socials.to_crs({'init':'epsg:2006'})
    return socials

#social_merged.crs = buildings.crs
build_soc_points = geopandas.sjoin(buildings, social_point_all, how='inner', op='intersects')
build_soc_points

social_point_ids = []

#social_point_ids = ['stluc_supermarkets']
#social_point_ids = ['Fire_Stations']
#social_point_ids = ['stluc_schools2018']
#'hotels','stluc_supermarkets', 'Fire_Stations', 'stluc_schools2018'

len(social_point_df)

social_fac = ['health_centers', 'pharmacies']

social= []
for social_point_id in social_fac:
    social_fac_df = load_social(data_folder, social_point_id)
    social_gov_build['category'] = 'healthcare'
    social_gov_build = social_gov_build[[
        
    ]]
    social_gov_build_all.append(social_gov_build)

social = pd.concat(social_gov_build_all, axis=0, sort = False)
social
    

social_fac = ['health_centers', 'pharmacies']

social= []
for social_point_id in social_fac:
    social_fac_df = load_social(data_folder, social_point_id)
    social_gov_build = geopandas.sjoin(buildings, social_point_df, how='inner', op='intersects')
    social_gov_build['category'] = 'healthcare'
    social_gov_build_all.append(social_gov_build)

social_gov_build_all = pd.concat(social_gov_build_all, axis=0, sort = False)
social_gov_build_all
    

#social_gov_build_all = []
for social_point_id in social_point_ids:
    social_point_df = load_social(data_folder, social_point_id)
    social_point_df['category'] = 

